In [1]:
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import os 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import torch.nn.functional as F
import torchvision.utils as utils

In [2]:
def load_data(src):
    channels_src = 'channels.txt'
    with open(os.path.join(src, channels_src)) as f:
        channels = f.read().splitlines()

    mat = np.loadtxt(os.path.join(src, 'mat.txt')).astype(np.float32)
    mat = (mat / np.linalg.norm(mat, ord=1, axis=0))
    return mat, channels
src = r'Z:\vovam\Vova\data\matrices\8_25'
mat, channels = load_data(src)
x = xr.load_dataarray(r'C:\Users\leeatgen.WISMAIN\Desktop\Vova\all_data.nc').sel(fovs='Point5',channels=channels).values.astype(np.float32)

In [3]:
print(x.ndim)
if x.ndim != 4:
    print(x.ndim)
    x = x.reshape((1, *x.shape))

NameError: name 'x' is not defined

In [ ]:
seed = np.random.randint(1,1000000000)
torch.manual_seed(seed)
X = torch.tensor(x.astype(np.float32), requires_grad=False)
print(X.shape)
plt.imshow(X[0,0,:,:])

In [4]:
A = torch.tensor(mat, requires_grad=False).unsqueeze(-1).unsqueeze(-1).float()
X = torch.tensor(x.astype(np.float32), requires_grad=False)

## To-do list-

[x] - Make for loop for batching

[X] - check about class for the transformations

In [ ]:
class CancerDataset(Dataset):
    def __init__(self, data, batch_size, random_crop, initial_transform=None):
        super().__init__()
        if initial_transform:
            self.data = initial_transform(data)
        else:
            self.data = data
            
        self.batch_size = batch_size    
        self.random_crop = random_crop
        self.transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(30),
    T.RandomPerspective(),
    T.RandomCrop(size=random_crop),
    T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    T.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.01, 1), value=0, inplace=False)
])
    def __len__(self):
        return(len(self.data))

    def fetch(self):
        batch = []
        for _ in range(self.batch_size):
            fov = np.random.randint(self.data.shape[0])
            batch.append(self.transform(self.data[fov]))
        return torch.stack(batch)

In [ ]:
center_crop = 512
take_crop = T.Compose([T.CenterCrop(center_crop)])

ds = CancerDataset(X,8,256,take_crop)
#Y = F.conv2d(ds, A)

In [ ]:
XX = ds.fetch()

In [ ]:
asd = take_crop(X)

In [ ]:
asd.shape

In [ ]:
Y = F.conv2d(XX, A)
Y2 = F.conv2d(XX, A)

In [ ]:
Y.shape

In [5]:
x_all = xr.load_dataarray(r'C:\Users\leeatgen.WISMAIN\Desktop\Vova\all_data.nc').values.astype(np.float32)

In [6]:
X_all = torch.tensor(x_all.astype(np.float32), requires_grad=False)

In [7]:
class CancerDataset2(Dataset):
    def __init__(self, data, random_crop, initial_transform=None):
        super().__init__()
        if initial_transform:
            self.data = initial_transform(data)
        else:
            self.data = data
            
        self.batch_size = batch_size    
        self.random_crop = random_crop
        self.transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(30),
    T.RandomPerspective(),
    T.RandomCrop(size=random_crop),
    T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    T.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.01, 1), value=0, inplace=False)
])
    def __len__(self):
        return(len(self.data))

    def __getitem__(self,idx):
        return self.transform(self.data[idx])

In [9]:
center_crop = 512
take_crop = T.Compose([T.CenterCrop(center_crop)])
ds2 = CancerDataset2(X_all,8,256,take_crop)

In [13]:
dl = DataLoader(ds2,40)

In [14]:
for i in dl:
    print(i.shape)

torch.Size([38, 40, 256, 256])
